In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch_geometric.nn import MessagePassing
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
import os
import sys
notebook_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(notebook_dir, ".."))
sys.path.append(project_root)
# from gnn_model.train_model import train_model
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
save_dir = os.path.join(parent_dir, "final_messages", "projectile_trajectory_2d")
os.makedirs(save_dir, exist_ok=True)
save_path = os.path.join(save_dir, f"final_epoch_messages_projectile_trajectory_2d.pkl")


In [2]:
# To save the final messages during training
def save_messages(sim_name, model):
    """
    Saves the message storage dictionary inside "/final_messages/{sim_name}/".
    
    Parameters:
    - sim_name: Name of the simulation (used for subfolder and file naming)
    - model: Trained GNN model containing the message storage dictionary
    """
    current_dir = os.getcwd()
    repo_root = os.path.dirname(current_dir)    
    final_messages_dir = os.path.join(repo_root, "final_messages", sim_name)
    os.makedirs(final_messages_dir, exist_ok=True)
    save_path = os.path.join(final_messages_dir, f"final_epoch_messages_{sim_name}.pkl")
    with open(save_path, "wb") as f:
        pickle.dump(model.message_storage, f)
    return save_path


In [3]:
import numpy as np
import torch

def projectile_trajectory_2d(m, dt, theta, v0, g = 9.81, x0 = 0, y0 = 0, N=1, dim=2):
    """
    Calculates a 2D cartesian trajectory of a projectile of mass m launched at angle theta with initial velocity v0.  
    
        Parameters:
    theta (int or float): angle between projectile trajectory and x-axis at t=0s in degrees \\
    v0 (int or float): the intial velocity in m/s \\
    m (int or float): mass of the projectile in kg \\
    dt (int or float): time step for simulation in s \\
    g (int or float): gravity constant (standard is 9.81) in m/s^2 \\
    x0, y0 (int or float): initial position of ball in m \\

        Returns: 
    The output is a dictionary containing the {time, positions, velocities, masses} coordinates of the projectile
    """

    # Convert theta to radians
    theta_rad = np.radians(theta)

    # Initial velocity components
    vx0 = v0 * np.cos(theta_rad)
    vy0 = v0 * np.sin(theta_rad)

    # Time array
    t_max = (vy0 + np.sqrt(vy0**2+2*g*y0)) / g
    t = np.arange(0, t_max, dt)
    T = len(t)
    print(T)
    
    # Compute trajectory
    x = x0 + vx0 * t
    y = y0 + vy0 * t - 0.5 * g * t**2

    # Remove negative y values and make np array
    valid = y >= 0
    x, y, t = x[valid], y[valid], t[valid]
    trajectory = np.column_stack((x,y))

    vx = np.full_like(x, vx0)
    vy = vy0 - g * t
    trajectory_velocities = np.column_stack((vx,vy))
    
    trajectory_data = {
        "time": torch.arange(T, dtype=torch.float64),
        "positions": torch.zeros((T, N, dim), dtype=torch.float64),
        "velocities": torch.zeros((T, N, dim), dtype=torch.float64),
        "masses": torch.arange(N, dtype=torch.float64)
    }
    
    trajectory_data["time"] = torch.tensor(t, dtype=torch.float64)
    trajectory_data["positions"] = torch.tensor(trajectory[:, :].reshape(T, N, dim), dtype=torch.float64)
    trajectory_data["velocities"] = torch.tensor(trajectory_velocities[:, :].reshape(T, N, dim), dtype=torch.float64)
    trajectory_data["masses"] = torch.tensor(m, dtype=torch.float64)

    return trajectory_data


In [4]:
import torch
from torch_geometric.data import Data

def node_data_list(trajectory_dict, self_loop=True, complete_graph=True):
    """
    Converts a trajectory dictionary into a list of PyTorch Geometric `Data` objects 
    representing graph-structured data for a node-based learning task.

    Parameters:
    -----------
    trajectory_dict : dict
        A dictionary containing simulation data with the following keys:
        - "masses" (torch.Tensor): Mass values for the nodes (particles).
        - "time" (torch.Tensor): Time steps in the trajectory.
        - "positions" (list of torch.Tensor): List of position tensors at each time step.
        - "velocities" (list of torch.Tensor): List of velocity tensors at each time step.

    self_loop : bool, optional (default=True)
        If True, self-loops (edges from a node to itself) are included in the graph.

    complete_graph : bool, optional (default=True)
        If True, a fully connected graph is created where each node is connected to every other node.

    Returns:
    --------
    data_list : list of torch_geometric.data.Data
        A list of `Data` objects, each representing a graph at a given time step.
        Each `Data` object contains:
        - `x` (torch.Tensor): Node features with shape (N, num_features), where `num_features` includes
          position, velocity, mass, and time.
        - `y` (torch.Tensor): Target values representing velocity updates.
        - `edge_index` (torch.Tensor): Graph connectivity in COO format with shape (2, num_edges).

    Notes:
    ------
    - The function assumes a constant number of nodes (N) throughout the trajectory.
    - If `masses` is a scalar, it is broadcasted to all nodes.
    - Edge indices are stored in COO format (two-row tensor).

    Example:
    --------
    >>> trajectory_dict = {
    ...     "masses": torch.tensor(1.0),
    ...     "time": torch.arange(3),
    ...     "positions": [torch.rand(5, 2) for _ in range(3)],
    ...     "velocities": [torch.rand(5, 2) for _ in range(3)]
    ... }
    >>> graphs = node_data_list(trajectory_dict)
    >>> print(graphs[0])
    Data(x=[5, 6], y=[5, 2], edge_index=[2, 25])
    """

    data_list = []

    N = trajectory_dict["masses"].numel()
    # Ensure masses has the correct shape (N, 1)
    mass_data = trajectory_dict["masses"]

    # If masses is scalar (0-dimensional), repeat it for N particles
    if mass_data.dim() == 0:  # scalar case
        mass_data = mass_data.repeat(N, 1)  # shape (N, 1)

    # If masses is already a 1D tensor of shape (N,)
    elif mass_data.dim() == 1:  # 1D tensor case
        mass_data = mass_data.unsqueeze(1)  # shape (N, 1)
    
    for i in range(len(trajectory_dict["time"]) - 1):
        time_feature = trajectory_dict["time"][i].repeat(N,1)
        # print(time_feature.shape)
        # position_features = trajectory_dict["positions"][i]

        x_features = torch.cat([
            trajectory_dict["positions"][i],    # (N,dim)
            trajectory_dict["velocities"][i],   # (N, dim)
            mass_data,       # (N, 1)
            time_feature    # (N, 1)
        ], dim=1)   # tensor of shape (N, 2 + 2 + 1 + 1) + (N, 6) if dim = 2

        velocity_update = trajectory_dict["velocities"][i+1] - trajectory_dict["velocities"][i]
        acceleration = velocity_update / (trajectory_dict["time"][i+1] - trajectory_dict["time"][i])

        y_target = torch.tensor(acceleration, dtype=torch.float32)
        # y_target = torch.tensor(velocity_update, dtype=torch.float32)

        edge_list = []
        
        if self_loop:
            edge_list.extend([j,j] for j in range(N))
        
        if complete_graph:
            edge_list.extend([k,j] for k in range(N) for j in range(N) if k != j)           
   
        edge_index = torch.tensor(edge_list, dtype=torch.long).t().contiguous()        

        data_list.append(Data(x=x_features.float(), y=y_target, edge_index=edge_index))

    return data_list


In [5]:
traj_proj = projectile_trajectory_2d(3, 0.2, 40, 90)
# print(traj_proj["time"].shape)

59


In [6]:
traj_graph = node_data_list(traj_proj, self_loop=True, complete_graph=False)

/var/folders/_p/ztk2c0b93nbb9q83wg0_pq780000gn/T/ipykernel_8305/1023136190.py:82: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_target = torch.tensor(acceleration, dtype=torch.float32)


In [7]:
### --- Jacob's code (doesn't work) --- ###

# import torch
# import torch.nn as nn
# import torch.optim as optim
# import numpy as np
# from torch_geometric.nn import MessagePassing
# from torch_geometric.data import Data
# from torch_geometric.loader import DataLoader

# # define the message passing class to initiate MLP's among the nodes
# class GNN_MLP(MessagePassing):
#     def __init__(self, n_f, m_dim, hidden_channels, out_channels, single_node = False):
#         super(GNN_MLP, self).__init__(aggr='add')  # "Add" aggregation for summing over forces
        
#         # initialising the MLP by creating the self.MLP attribute. 2 * in_channels to account for the fact that it may use both it's own and the other nodes features.
#         self.mess_mlp = nn.Sequential(
#             nn.Linear(2 * n_f, hidden_channels),
#             nn.ReLU(),
#             nn.Linear(hidden_channels, hidden_channels),
#             nn.ReLU(),
#             nn.Linear(hidden_channels, m_dim)
#         )

#         self.agg_mlp = nn.Sequential(
#             nn.Linear(m_dim + n_f, hidden_channels),
#             nn.ReLU(),
#             nn.Linear(hidden_channels, hidden_channels),
#             nn.ReLU(),
#             nn.Linear(hidden_channels, out_channels)
#         )
        
#         self.single_node = single_node
#         self.message_storage = {}
#         self.store_messages = False  

#     def forward(self, x, edge_index, final_epoch=False):
#         """
#         Forward calls propagate to initiate message passing for all nodes in edge_index
#         """
#         self.store_messages = final_epoch
#         return self.propagate(edge_index, size=(x.size(0), x.size(0)), x=x)  # Triggers message passing
    
#     def message(self, x_i, x_j, edge_index):
#         """
#         Applying MLP to every directed edge in edge_index for [x_i, x_j].
#         Also stores positions, times, and velocities for final epoch.
#         """
#         edge_features = torch.cat([x_i, x_j], dim=1)  # Concatenate node features
#         messages = self.mess_mlp(edge_features)  # Compute messages

#         if self.store_messages:  # Store messages in final epoch
#             edges = list(zip(edge_index[0].tolist(), edge_index[1].tolist()))

#             for (src, dest), msg, x_src, x_dest in zip(edges, messages, x_i, x_j):
#                 # Extract physical attributes from feature vector x_i, x_j
#                 position_src, position_dest = x_src[:3].cpu().numpy(), x_dest[:3].cpu().numpy()
#                 velocity_src, velocity_dest = x_src[3:6].cpu().numpy(), x_dest[3:6].cpu().numpy()
#                 time_src, time_dest = x_src[6].item(), x_dest[6].item()  # Assuming time is 7th feature

#                 # Store structured message data
#                 self.message_storage[(src, dest)] = {
#                     "message": msg.detach().cpu().numpy(),
#                     "position_src": position_src,
#                     "position_dest": position_dest,
#                     "velocity_src": velocity_src,
#                     "velocity_dest": velocity_dest,
#                     "time_src": time_src,
#                     "time_dest": time_dest
#                 }

#         return messages  # Pass messages to aggregation

#     def update(self, aggr_out, x=None):
#         """
#         Updates node features with passed messages.
#         """
#         if self.single_node:
#             return aggr_out
        
#         else:
#             update_features = torch.cat([x, aggr_out], dim=1)
#             return self.agg_mlp(update_features)

In [8]:
# ### --- This works --- ###

# import torch
# import torch.nn as nn
# import torch.optim as optim
# import numpy as np
# from torch_geometric.nn import MessagePassing
# from torch_geometric.data import Data
# from torch_geometric.loader import DataLoader

# # define the message passing class to initiate MLP's among the nodes
# class GNN_MLP(MessagePassing):
#     def __init__(self, n_f, m_dim, hidden_channels, out_channels, single_node = False):
#         super(GNN_MLP, self).__init__(aggr='add')  # "Add" aggregation for summing over forces
        
#         # initialising the MLP by creating the self.MLP attribute. 2 * in_channels to account for the fact that it may use both it's own and the other nodes features.
#         self.mess_mlp = nn.Sequential(
#             nn.Linear(2 * n_f, hidden_channels),
#             nn.ReLU(),
#             nn.Linear(hidden_channels, hidden_channels),
#             nn.ReLU(),
#             nn.Linear(hidden_channels, m_dim)
#         )

#         self.agg_mlp = nn.Sequential(
#             nn.Linear(m_dim + n_f, hidden_channels),
#             nn.ReLU(),
#             nn.Linear(hidden_channels, hidden_channels),
#             nn.ReLU(),
#             nn.Linear(hidden_channels, out_channels)
#         )
        
#         self.single_node = single_node
#         self.message_storage = {}
#         self.store_messages = False  

#     def forward(self, x, edge_index, final_epoch=False):
#         """
#         Forward calls propagate to initiate message passing for all nodes in edge_index
#         """
#         self.store_messages = final_epoch
#         return self.propagate(edge_index, size=(x.size(0), x.size(0)), x=x)  # Triggers message passing
    
#     def message(self, x_i, x_j, edge_index):
#         """
#         Applying mlp to every directed edge in edge_index for [x_i, x_j]
#         """
#         edge_features = torch.cat([x_i, x_j], dim=1)  # Concatenating node features for edge
#         messages = self.mess_mlp(edge_features)

#         if self.store_messages: # Store messages for final epoch
#             edges = list(zip(edge_index[0].tolist(), edge_index[1].tolist()))
#             for e, msg in zip(edges, messages):
#                 self.message_storage[e] = msg.detach().cpu().numpy() # detach from tree, found a new favourite
#         return messages  # Pass through MLP

#     def update(self, aggr_out, x=None):
#         """
#         Updates node features with passed messages.
#         """
#         if self.single_node:
#             return aggr_out
        
#         else:
#             update_features = torch.cat([x, aggr_out], dim=1)
#             return self.agg_mlp(update_features)

In [9]:
class GNN_MLP(MessagePassing):
    def __init__(self, n_f, m_dim, hidden_channels, out_channels, single_node=False):
        super(GNN_MLP, self).__init__(aggr='add')
        
        self.mess_mlp = nn.Sequential(
            nn.Linear(2 * n_f, hidden_channels),
            nn.ReLU(),
            nn.Linear(hidden_channels, hidden_channels),
            nn.ReLU(),
            nn.Linear(hidden_channels, m_dim)
        )

        self.agg_mlp = nn.Sequential(
            nn.Linear(m_dim + n_f, hidden_channels),
            nn.ReLU(),
            nn.Linear(hidden_channels, hidden_channels),
            nn.ReLU(),
            nn.Linear(hidden_channels, out_channels)
        )
        
        self.single_node = single_node
        self.message_storage = []
        self.store_messages = False
        self.current_time = None

    def forward(self, x, edge_index, pos=None, final_epoch=False):
        self.store_messages = final_epoch
        # Extract the time from the last column of x (assumes time is broadcasted to all nodes)
        if final_epoch:
            #self.current_time = round(x[0, -1].item(), 4) # Just take it from the first node
            self.current_time = x[0, -1].item()

        return self.propagate(edge_index, size=(x.size(0), x.size(0)), x=x, pos=pos)

    def message(self, x_i, x_j, edge_index_i, edge_index_j, pos_i=None, pos_j=None):
        edge_features = torch.cat([x_i, x_j], dim=1)
        messages = self.mess_mlp(edge_features)

        if self.store_messages:
            for i in range(messages.size(0)):
                record = {
                    'edge': (edge_index_i[i].item(), edge_index_j[i].item()),
                    'message': messages[i].detach().cpu().numpy(),
                    'pos_i': pos_i[i].detach().cpu().numpy() if pos_i is not None else None,
                    'pos_j': pos_j[i].detach().cpu().numpy() if pos_j is not None else None,
                    'time': self.current_time
                }
                self.message_storage.append(record)

        return messages

    def update(self, aggr_out, x=None):
        if self.single_node:
            return aggr_out
        else:
            update_features = torch.cat([x, aggr_out], dim=1)
            return self.agg_mlp(update_features)


In [10]:
import numpy as np
import torch
import torch.optim as optim
import torch_geometric
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
import pickle
import os
current_dir = os.getcwd()
# Move one level up (assuming your notebook is inside "Test" and "final_messages" is in the parent)
parent_dir = os.path.dirname(current_dir)

# Define the correct path to the `final_messages` folder

def train_model(model, train_data, sim_name:str, epochs=100, lr=0.01):
    """
    Train a GNN model using the provided training and validation loaders.
    
    Parameters:
    -----------
    model : torch.nn.Module
        A PyTorch module representing the GNN model.
    train_loader : torch_geometric.loader.DataLoader
        A DataLoader object containing training data.
    val_loader : torch_geometric.loader.DataLoader
        A DataLoader object containing validation data.
    epochs : int
        Number of epochs to train the model.
    lr : float
        Learning rate for the optimizer.
    
    Returns:
    --------
    model : torch.nn.Module
        The trained model.
    """
    
    # Only convert to DataLoader if not already in DataLoader format
    if isinstance(train_data, DataLoader):
        train_data = DataLoader(train_data, batch_size=1, shuffle=False)

    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = torch.nn.MSELoss()
    
    for epoch in range(epochs):
        total_loss = 0
        final_epoch = (epoch == epochs - 1)
        for data in train_data:
            optimizer.zero_grad()
            out = model(data.x, data.edge_index, final_epoch=final_epoch)
            loss = criterion(out, data.y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
    save_messages(sim_name, model)
    
    return model


In [11]:
model = GNN_MLP(n_f=6, m_dim=2, out_channels=2, hidden_channels=128, single_node=True)
train_model(model, traj_graph, lr=0.01, epochs=100, sim_name="projectile_trajectory_2d")

GNN_MLP()

In [12]:
# Output results
print("\nSimulation Results:")
for idx, data in enumerate(traj_graph):
    with torch.no_grad():
        pred_force = model(data.x, data.edge_index)  # Model predicts net force
        
        # Ground truth force
        ground_truth_force = data.y
        
        # Ensure correct shapes and conversion to NumPy for printing
        pred_force_np = pred_force.cpu().numpy().flatten()  # Convert to NumPy array and flatten
        ground_truth_np = ground_truth_force.cpu().numpy().flatten()  # Convert to NumPy and flatten

        # Print each component of the force vector
        print(f"Sample {idx + 1}:")
        for i in range(len(pred_force_np)):
            print(f"   Component {i}: Ground Truth = {ground_truth_np[i]:.6f}, Predicted = {pred_force_np[i]:.6f}")



Simulation Results:
Sample 1:
   Component 0: Ground Truth = 0.000000, Predicted = 0.116828
   Component 1: Ground Truth = -9.810000, Predicted = -9.329630
Sample 2:
   Component 0: Ground Truth = 0.000000, Predicted = 0.109364
   Component 1: Ground Truth = -9.810000, Predicted = -9.360141
Sample 3:
   Component 0: Ground Truth = 0.000000, Predicted = 0.102027
   Component 1: Ground Truth = -9.810000, Predicted = -9.390607
Sample 4:
   Component 0: Ground Truth = 0.000000, Predicted = 0.092422
   Component 1: Ground Truth = -9.810000, Predicted = -9.400360
Sample 5:
   Component 0: Ground Truth = 0.000000, Predicted = 0.082732
   Component 1: Ground Truth = -9.810000, Predicted = -9.408239
Sample 6:
   Component 0: Ground Truth = 0.000000, Predicted = 0.073144
   Component 1: Ground Truth = -9.810000, Predicted = -9.415851
Sample 7:
   Component 0: Ground Truth = 0.000000, Predicted = 0.063063
   Component 1: Ground Truth = -9.810000, Predicted = -9.423305
Sample 8:
   Component 0: G

In [14]:
# Forward pass (with final_epoch=True to enable message saving)
output = model(x=data.x, edge_index=data.edge_index, pos=data.pos if hasattr(data, "pos") else None, final_epoch=True)

# Print stored messages
print("\nStored Messages:")
for i, record in enumerate(model.message_storage):
    print(f"Message {i + 1}")
    print(f"  Edge        : {record['edge']}")
    print(f"  Message     : {record['message']}")
    print(f"  pos_i       : {record['pos_i']}")
    print(f"  pos_j       : {record['pos_j']}")
    print(f"  Time        : {record['time']}")
    print("-" * 40)



Stored Messages:
Message 1
  Edge        : (0, 0)
  Message     : [  0.107056 -10.854086]
  pos_i       : None
  pos_j       : None
  Time        : 0.0
----------------------------------------
Message 2
  Edge        : (0, 0)
  Message     : [ -0.0675264 -10.05479  ]
  pos_i       : None
  pos_j       : None
  Time        : 0.20000000298023224
----------------------------------------
Message 3
  Edge        : (0, 0)
  Message     : [-0.10182624 -9.241906  ]
  pos_i       : None
  pos_j       : None
  Time        : 0.4000000059604645
----------------------------------------
Message 4
  Edge        : (0, 0)
  Message     : [ 5.4776669e-05 -8.8701725e+00]
  pos_i       : None
  pos_j       : None
  Time        : 0.6000000238418579
----------------------------------------
Message 5
  Edge        : (0, 0)
  Message     : [ 0.07705088 -9.043863  ]
  pos_i       : None
  pos_j       : None
  Time        : 0.800000011920929
----------------------------------------
Message 6
  Edge        : (0